# Development

Agreement and policy: https://developer.twitter.com/en/developer-terms/agreement-and-policy

In [ ]:
import os
os.chdir('..')

from modules import TwitterDataCleaner, get_tweets

## Get tweets

In [ ]:
query = 'sandra torres'
start_time = "2023-04-26T00:00:00Z" 
end_time = "2023-04-30T00:00:00Z" 
max_results = 10

tweets_call = get_tweets(
    query=query,
    start_time=start_time,
    end_time=end_time,
    max_results=max_results,
)

data_cleaner = TwitterDataCleaner(tweets_call)
tweets, users = data_cleaner.run()

In [ ]:
tweets.head()

## Zero-shot feature extraction with GTP-3.5

In [ ]:
import re
import json
import openai
import pandas as pd
from authenticators import OpenAIAuthenticator

gpt = OpenAIAuthenticator()
openai.api_key = gpt.api_key

In [ ]:
def preprocess_text(text):
    # Remove URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = url_pattern.sub('', text)

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub('', text)

    # Remove newline characters
    text = text.replace('\n', ' ')

    return text

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [ ]:
example = tweets.tw_texto[9]

tweet = preprocess_text(example)
tweet

In [ ]:
prompt = f"""
El siguiente es un tweet en español. Por favor, clasificalo de acuerdo a las siguientes categorías:

Valencia: [positivo, negativo, neutral]
Emoción: [felicidad, tristeza, enojo, miedo, sorpresa, disgusto, neutral]
Postura: [aprobación, desaprobación, esperanza, desilusión, indiferencia, condianza, desconfianza]
Tono: [agresivo, pasivo, asertivo, escéptico, irónico, humorístico, informativo, serio, inspiridor, otro]
Tema: [política, economía, deportes, entretenimiento, tecnología, ciencia, salud, educación, religión, cultura, medio ambiente, otro]

Además, evalúalo utilizando una escala continua con rango de 0 a 1 en las siguientes dimensiones:

Amabilidad: [0.0 - 1.0]
Legibilidad: [0.0 - 1.0]
Controversialidad: [0.0 - 1.0]
Informatividad: [0.0 - 1.0]
Espectro político: [0.0 - 1.0] (0 = izquierda, 1 = derecha)

Formatea tu respuesta como un diccionario de Python con las siguientes llaves:

[valencia, emocion, postura, tono, tema, amabilidad, legibilidad, controversialidad, informatividad]

Tweet: '''{tweet}'''
"""

In [ ]:
test = get_completion(prompt, temperature=0)
dict = json.loads(test)
dict

In [ ]:
pd.DataFrame(dict, index=[0])